In [ ]:
import os

xml = os.listdir('VOCdevkit/VOC2007/Annotations')
train = open('VOCdevkit/VOC2007/ImageSets/Main/train.txt', 'w')

for i in range(0,len(xml)):
    name = xml[i][:-4] + '\n'
    train.write(name)

train.close()

#自己制作的数据集需要运行，生成Main目录下的train.txt文件，VOC官方数据集已自带train.txt文件，不需要执行。
#参考：https://blog.csdn.net/tzwsg/article/details/107116132

In [ ]:
sets=[('2007', 'trainval')]
classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

# sets=[('2007', 'train')]
# classes = ["airplane"]

#自己制作的数据集需要运行下面两行，VOC官方数据集运行上面两行。

In [ ]:
import xml.etree.ElementTree as ET
from os import getcwd

def convert_annotation(year, image_id, list_file):
    in_file = open('VOCdevkit/VOC%s/Annotations/%s.xml'%(year, image_id))
    tree=ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))
        
wd = getcwd()

for year, image_set in sets:
    image_ids = open('VOCdevkit/VOC%s/ImageSets/Main/%s.txt'%(year, image_set)).read().strip().split()
    list_file = open('%s_%s.txt'%(year, image_set), 'w')
    for image_id in image_ids:
        list_file.write('%s/VOCdevkit/VOC%s/JPEGImages/%s.jpg'%(wd, year, image_id))
        convert_annotation(year, image_id, list_file)
        list_file.write('\n')
    list_file.close()
    
#生成图片路径+标签信息的txt，用于YOLOv3后续载入数据集使用。
#参考：https://github.com/qqwweee/keras-yolo3/blob/master/voc_annotation.py